# Metabolic network analysis using *volestipy*

## Dependencies

Before showing how you can exploit the *volestipy* software, we first need to get all the relative dependencies. 

This demo uses [Anaconda](https://www.anaconda.com/products/individual) which you can download following [these](https://www.digitalocean.com/community/tutorials/how-to-install-anaconda-on-ubuntu-18-04-quickstart) instructions.

Furtheremore, special, powerful mathematical optimization solvers like [Gurobi](https://www.gurobi.com/) are also used. You can get Gurobi following the steps described [here](https://support.gurobi.com/hc/en-us/articles/360044290292-Installing-Gurobi-for-Python).

To read a .mat file you need some extra Python libraries, as it is quite a challenging task. If you are more interested in that, you can read this article [here](https://scipy-cookbook.readthedocs.io/items/Reading_mat_files.html).

To get any libraries that need to run commands as ```sudo``` you need to make a file including **only** your password and replace ```/home/haris/Desktop/running/metabolic_network_pipeline_volestipy/my_project_virtual_env/error.txt``` with the corresponding path. 

In [1]:
print("hello friend")
import getpass
import os

hello friend


In [2]:
# Get the ggplot - oriented Python library
!sudo -H -S pip3 install pandas plotnine < /home/haris/Desktop/running/metabolic_network_pipeline_volestipy/my_project_virtual_env/error.txt
print("*** The ggplot for Python library has now been installed *** \n\n")

# Get the tables library to read .mat files
!sudo -H -S pip3 install tables < /home/haris/Desktop/running/metabolic_network_pipeline_volestipy/my_project_virtual_env/error.txt
print("*** The tables library has now been installed *** \n\n")

# Get the h5py library in case you are working with .mat files of 7.3 release of Matlab and after 
!sudo -H -S pip3 install h5py < /home/haris/Desktop/running/metabolic_network_pipeline_volestipy/my_project_virtual_env/error.txt
print("*** The h5py library has now been installed *** \n\n")

# Get GUROBI through anaconda - You can find more about installing Gurobi here: https://support.gurobi.com/hc/en-us/articles/360044290292-Installing-Gurobi-for-Python
!conda install -y -c gurobi gurobi
print("*** The Gurobi solver library has now been installed *** \n\n")

[sudo] password for haris: Requirement already satisfied: pandas in /usr/local/lib/python3.6/dist-packages (1.0.3)
[sudo] password for haris: Keyring is skipped due to an exception: org.freedesktop.DBus.Error.NoServer: Failed to connect to socket /tmp/dbus-L8VEqumkCi: Connection refused
*** The tables library has now been installed *** 


[sudo] password for haris: Keyring is skipped due to an exception: org.freedesktop.DBus.Error.NoServer: Failed to connect to socket /tmp/dbus-L8VEqumkCi: Connection refused
*** The h5py library has now been installed *** 


Solving environment: done

# All requested packages already installed.

*** The Gurobi solver library has now been installed *** 




In [5]:
import gurobipy as gp
from gurobipy import GRB

In [7]:
# Create a new model
m = gp.Model("mip1")

# Create variables
x = m.addVar(vtype=GRB.BINARY, name="x")
y = m.addVar(vtype=GRB.BINARY, name="y")
z = m.addVar(vtype=GRB.BINARY, name="z")

# Set objective
m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

# Add constraint: x + 2 y + 3 z <= 4
m.addConstr(x + 2 * y + 3 * z <= 4, "c0")

# Add constraint: x + y >= 1
m.addConstr(x + y >= 1, "c1")

# Optimize model
m.optimize()

for v in m.getVars():
    print('%s %g' % (v.varName, v.x))

print('Obj: %g' % m.objVal)


Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0xf43f5bdf
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.03s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.05 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
x 1
y 0
z 1
Obj: 3


Now we can import all the necessary libraries.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from plotnine import *
%matplotlib inline

# Matlab up to 7.1 = mat files created with Matlab up to version 7.1 can be read using the mio module part of scipy.io.
from scipy.io import loadmat 
# Beginning at release 7.3 of Matlab, mat files are actually saved using the HDF5 format by default (except if you use the -vX flag at save time, see in Matlab). These files can be read in Python using, for instance, the PyTables or h5py package
import tables 
import h5py

The following command is recommended for mat files created with Matlab up to version 7.1 c

## Read your network file

In [ ]:
mat_file_with_loadmat = loadmat('/home/haris/Downloads/e_coli_core.mat')

While this one is in case of ```.mat``` files created by releases of Matlab later after the 7.3
This command will not run in any other case.
For this demo we will use the ```loadmat``` option.

In [ ]:
# mat_file_with_tables = h5py.File('/home/haris/Downloads/e_coli_core.mat')

Now you can see your metabolic network. 

In [ ]:
print(mat_file_with_loadmat)

In [ ]:
data_from_mat = mat_file_with_loadmat
print("the data type of the variable with the network as it was read is: " + str(type(data_from_mat)))

s_matrix = data_from_mat.keys()
print("\nthe keys of this dictionaries are: ")
print(s_matrix)

e_coli_np_void = data_from_mat['e_coli_core'][0][0]
print("\nHowever, if we keep the key:value pair of this dictionary, called 'e_coli_core' where the necessary \
information is located, we can see that its type is: " + str(type(e_coli)) + "\n\n")

print("number of dimensions of the np.void data type equals to:" + str(e_coli_np_void.ndim) + "\n")

print(type(e_coli_np_void))
print(len(e_coli_np_void))



# metabolites
print(type(e_coli_np_void[0]))
print(e_coli_np_void[0].shape)

print(type(e_coli_np_void[0][0]))
print(e_coli_np_void[0][0].shape)

print(e_coli_np_void[0][:3,])
metabolites = [item[0][0] for item in e_coli_np_void[0]]
print(metabolites)

# genes
print(type(e_coli_np_void[4]))
print(e_coli_np_void[4].shape)
print(e_coli_np_void[4][:3,])

# reactions
print(type(e_coli_np_void[7]))
print(e_coli_np_void[7].shape)
print(e_coli_np_void[7][:3,])



# print(type(e_coli_np_void[5]))
# print(e_coli_np_void[5].ndim)
# print(e_coli_np_void[5][:3,])

# print(type(e_coli_np_void[6]))
# print(e_coli_np_void[6].ndim)
# print(e_coli_np_void[6][:3,])



# print(type(e_coli_np_void[8]))
# print(e_coli_np_void[8].ndim)
# print(e_coli_np_void[8][:3,])

# print(type(e_coli_np_void[9]))
# print(e_coli_np_void[9].ndim)
# print(e_coli_np_void[9][:3,])

# print(type(e_coli_np_void[25]))
# print(e_coli_np_void[25].ndim)
# print(e_coli_np_void[25][:3,])


In [ ]:
counter = 0


# for entry in data_from_mat['e_coli_core'][0][0]:
#     print(entry[0])
#     counter += 1
#     print(counter)

In [ ]:
for key,value in data_from_mat.items():
    print(str(key) + "\t" + str(value))
    print("\n\n\n\n\n\n")

## Preprocess

## Full dimensional (not always required step)

## Rounding

## Sampling

## Analysis, plots etc.

In [ ]:
import sys
sys.version_info